In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/human-annotation/human_annotated_tweets.csv


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification
import tensorflow as tf


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
df = pd.read_csv('/kaggle/input/human-annotation/human_annotated_tweets.csv')

In [4]:
df

,text,url,manifestolabel_human
0,"Strengere einheitliche Regeln hinzubekommen, u...",https://twitter.com/2857610675/status/13360661...,welfare +
1,"Alles, was gesagt wurde, sind nur Mittel, und ...",https://twitter.com/1307748781945954306/status...,traditional morality +
2,"Nr. 177\nMan hört, Sie erkundigten sich nach I...",https://twitter.com/1185108027994714112/status...,political authority +
3,"Mehr Leute sollten sich trauen, sich öffentlic...",https://twitter.com/1134270667661217794/status...,social justice +
4,"Wie kann es sein, dass der Staat dem #Mittelst...",https://twitter.com/910913583168573440/status/...,market regulation +
...,...,...,...
5541,Das Verbrennen von #Holz und Pellets ist nicht...,https://twitter.com/robin_wood/status/12598249...,environmentalism +
5542,#Merz zu Ende gedacht: Damit die jüngere Gener...,https://twitter.com/UlrichSchneider/status/123...,welfare +
5543,Kommt ihr mit auf einen Spaziergang im Schnee ...,https://twitter.com/HHguidemelanie/status/1355...,undefined
5544,Eben fuhr ein ca. 3 jähriger in einem Mercedes...,https://twitter.com/demutsch/status/1256161453...,undefined


In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import TFXLMRobertaForSequenceClassification, XLMRobertaTokenizer

# Load data


# Encode labels
le = LabelEncoder()
df['labels'] = le.fit_transform(df['manifestolabel_human'])


In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import TFXLMRobertaForSequenceClassification, XLMRobertaTokenizer






# Tokenize tweets
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
inputs = tokenizer(df['text'].to_list(), max_length=512, truncation=True, padding=True, return_tensors='tf')

# Extract inputs and labels
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
labels = df['labels'].to_numpy()

# Split data into train and test sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids.numpy(), labels, test_size=0.2, random_state=42)

# Convert them into TensorFlow tensors
train_inputs = tf.convert_to_tensor(train_inputs)
val_inputs = tf.convert_to_tensor(val_inputs)
train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)

# Prepare TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': train_inputs}, train_labels)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': val_inputs}, val_labels)).batch(16)

# Load model
model = TFXLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=len(le.classes_))

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train model
model.fit(train_dataset, epochs=10, validation_data=val_dataset)


All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
278/278 [==============================] - 237s 661ms/step - loss: 3.1941 - accuracy: 0.2090 - val_loss: 3.0574 - val_accuracy: 0.2288
Epoch 2/10
278/278 [==============================] - 181s 650ms/step - loss: 3.0839 - accuracy: 0.2182 - val_loss: 3.0587 - val_accuracy: 0.2288
Epoch 3/10
278/278 [==============================] - 181s 652ms/step - loss: 3.0827 - accuracy: 0.2182 - val_loss: 3.0675 - val_accuracy: 0.2288
Epoch 4/10
278/278 [==============================] - 181s 653ms/step - loss: 3.0816 - accuracy: 0.2182 - val_loss: 3.0587 - val_accuracy: 0.2288
Epoch 5/10
278/278 [==============================] - 181s 651ms/step - loss: 3.0847 - accuracy: 0.2182 - val_loss: 3.0590 - val_accuracy: 0.2288
Epoch 6/10
278/278 [==============================] - 181s 652ms/step - loss: 3.0850 - accuracy: 0.2182 - val_loss: 3.0645 - val_accuracy: 0.2288
Epoch 7/10
278/278 [==============================] - 181s 653ms/step - loss: 3.0789 - accuracy: 0.2182 - val_loss: 3.0579 -

In [ ]:
from transformers import TFXLMRobertaForSequenceClassification, XLMRobertaTokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import tensorflow as tf

# Assuming df and le.classes_ exist in your workspace

# Tokenize tweets
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
inputs = tokenizer(df['text'].to_list(), max_length=512, truncation=True, padding=True, return_tensors='tf')

# Extract inputs and labels
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
labels = df['labels'].to_numpy()

# Split data into train and test sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids.numpy(), labels, test_size=0.2, random_state=42)

# Convert them into TensorFlow tensors
train_inputs = tf.convert_to_tensor(train_inputs)
val_inputs = tf.convert_to_tensor(val_inputs)
train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)

# Prepare TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': train_inputs}, train_labels)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': val_inputs}, val_labels)).batch(16)

# Load model
model = TFXLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=len(le.classes_))

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Define Early Stopping and Learning Rate Reduction on Plateau callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)

# Train model
model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[early_stopping, lr_scheduler])
